In [1]:
# Set local python and nltk paths
import sys
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/IPython/extensions')
sys.path

['/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/IPython/extensions',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages',
 '',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python27.zip',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/plat-linux2',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/lib-tk',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/lib-old',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/lib-dynload',
 '/homes/ag6516/.local/lib/python2.7/site-packages',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python

In [2]:
%matplotlib inline

import os
import pickle

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import skimage.transform
import random
from random import randint
random.seed(123)
np.random.seed(123)

# Set THEANO_FLAGS='device=cuda0,floatX=float32' to run notebook on gpu
import lasagne
from lasagne.utils import floatX
import theano
import theano.tensor as T
from collections import Counter
import tensorflow as tf
import googlenet

ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 164, in <module>
    use(config.device)
  File "/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 151, in use
    init_dev(device)
  File "/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 60, in init_dev
    sched=config.gpuarray.sched)
  File "pygpu/gpuarray.pyx", line 634, in pygpu.gpuarray.init
  File "pygpu/gpuarray.pyx", line 584, in pygpu.gpuarray.pygpu_init
  File "pygpu/gpuarray.pyx", line 1057, in pygpu.gpuarray.GpuContext.__cinit__
GpuArrayException: cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [3]:
dir = ('/vol/medic02/users/ag6516/x_ray_fracture_localisation/')
# dir = ('/Users/Aydan/PhD/x_ray_fracture_localisation/')

In [4]:
sample_exams = pickle.load(open('sample_exams_train_test_split.pkl'))
#sample_exams = pickle.load(open('augmented_exams_bbox_features.pkl'))
print len(sample_exams)

327


In [5]:
allwords = Counter()
for exam in sample_exams:
    tokens = exam['tokens']
    #for sentence in item['sentences']:
    allwords.update(tokens)
        
vocab = [k for k, v in allwords.items()]
vocab.insert(0, '#START#')
vocab.append('#END#')

word_to_index = {w: i for i, w in enumerate(vocab)}
index_to_word = {i: w for i, w in enumerate(vocab)}

len(vocab)

167

## Extract CNN Features from GoogLeNet

In [6]:
cnn_layers = googlenet.build_model()
cnn_input_var = cnn_layers['input'].input_var
cnn_feature_layer = cnn_layers['pool5/7x7_s1']
cnn_output_layer = cnn_layers['prob']

get_cnn_features = theano.function([cnn_input_var], lasagne.layers.get_output(cnn_feature_layer))

In [7]:
model_param_values = pickle.load(open('models/blvc_googlenet.pkl'))['param values']
lasagne.layers.set_all_param_values(cnn_output_layer, model_param_values)

In [8]:
MEAN_VALUES = np.array([104, 117, 123]).reshape((3,1,1))

def prep_image(im):
    if len(im.shape) == 2:
        im = im[:, :, np.newaxis]
        im = np.repeat(im, 3, axis=2)
    # Resize so smallest dim = 256, preserving aspect ratio
    h, w, _ = im.shape
    if h < w:
        im = skimage.transform.resize(im, (256, w*256/h), preserve_range=True)
    else:
        im = skimage.transform.resize(im, (h*256/w, 256), preserve_range=True)

    # Random crop to 224x224
    h_, w_, _ = im.shape
    #print h_, w_
    #print h_, w_
    #im = im[h//2-112:h//2+112, w//2-112:w//2+112]
    max_dx = h_-224-1
    max_dy = w_-224-1
    #print 'Max rand int: ', max_dx, max_dy
    dx = random.randint(0, max_dx)
    dy = random.randint(0, max_dy)
    #print 'dx, dy: ', dx, dy
    im = im[dx:dx+224, dy:dy+224, :]
    #print 'Cropped shape: ', im.shape

    rawim = np.copy(im).astype('uint8')
    #plt.figure()
    #plt.imshow(rawim)
    # Shuffle axes to c01
    im = np.swapaxes(np.swapaxes(im, 1, 2), 0, 1)
    
    # Convert to BGR
    im = im[::-1, :, :]

    im = im - MEAN_VALUES
    return rawim, floatX(im[np.newaxis])

### Augment dataset, extract features

In [9]:
def shuffle_sentences(tokens):
    if len(tokens) > 1:
        if tokens[0]== '.':
            tokens.pop(0)
        report = ' '.join(tokens)
        #print report
        sentences = report.split('.')
        #print sentences
        random.shuffle(sentences)
        #print sentences
        shuffled_sentences = ' . '.join(sentences)
        shuffled_sentences = shuffled_sentences.split(' ')
        s_sentences = [tok for tok in shuffled_sentences if tok!='']
        if len(s_sentences) > 1:
            if s_sentences[0]== '.':
                s_sentences.pop(0)
            if s_sentences[-1]!= '.':
                s_sentences.append('.')
    else:
        s_sentences = tokens
    return s_sentences

In [10]:
augmented_exams_set = pickle.load(open('augmented_images_baseline.pkl'))
print len(augmented_exams_set)
loaded_ids = []
for exam in augmented_exams_set:
    loaded_ids.append(exam['patient id'])
loaded_ids = set(loaded_ids)
print len(loaded_ids)

2154
327


In [ ]:
loaded_ids = []
augmented_exams_set = []
for exam in sample_exams:
    if exam['patient id'] not in loaded_ids:
        print exam['patient id']
        if exam['split'] == 'train':

            new_exam0 = exam
            new_images0 = []
            
            new_exam1 = exam
            new_images1 = []

            new_exam2 = exam
            new_images2 = []

            new_exam3 = exam
            new_images3 = []

            for image in exam['images']:
                new_image0 = image
                new_image1 = image
                new_image2 = image
                new_image3 = image

                path = dir + 'data/Images/' + image['impath']
                #print path
                im = plt.imread(path)
                
                # Original
                _, cnn_input0 = prep_image(im)
                features0 = get_cnn_features(cnn_input0)
                #print features.shape
                new_image0['cnn features'] = features0
                new_images0.append(new_image0)


                # Mirror flip
                im_flip1 = np.flip(im, 1)
                _, cnn_input1 = prep_image(im_flip1)
                features1 = get_cnn_features(cnn_input1)
                #print features.shape
                new_image1['cnn features'] = features1
                new_images1.append(new_image1)

                # Random Cropping
                _, cnn_input2 = prep_image(im)
                features2 = get_cnn_features(cnn_input2)
                #print features.shape
                new_image2['cnn features'] = features2
                new_images2.append(new_image2)

                # Mirror flip
                im_flip3 = np.flip(im, 1)
                _, cnn_input3 = prep_image(im_flip3)
                features3 = get_cnn_features(cnn_input3)
                #print features.shape
                new_image3['cnn features'] = features3
                new_images3.append(new_image3)

            # image transformations
            new_exam0['images'] = new_images0
            augmented_exams_set.append(new_exam0)
            new_exam1['images'] = new_images1
            augmented_exams_set.append(new_exam1)
            new_exam2['images'] = new_images2
            augmented_exams_set.append(new_exam2)
            new_exam3['images'] = new_images3
            augmented_exams_set.append(new_exam3)

            # sentence shuffling
            new_exam00 = new_exam0
            new_exam11 = new_exam1
            new_exam22 = new_exam2
            new_exam33 = new_exam3

            new_exam00['tokens'] = shuffle_sentences(exam['tokens'])
            augmented_exams_set.append(new_exam00)
            new_exam11['tokens'] = shuffle_sentences(exam['tokens'])
            augmented_exams_set.append(new_exam11)
            new_exam22['tokens'] = shuffle_sentences(exam['tokens'])
            augmented_exams_set.append(new_exam22)
            new_exam33['tokens'] = shuffle_sentences(exam['tokens'])
            augmented_exams_set.append(new_exam33)

            print len(augmented_exams_set)
        else:
            new_exam = exam
            new_images = []
            
            for image in exam['images']:
                new_image = image
                path = dir + 'data/Images/' + image['impath']
                #print path
                im = plt.imread(path)
                _, cnn_input = prep_image(im)
                features = get_cnn_features(cnn_input)
                #print features.shape
                new_image['cnn features'] = features
                new_images.append(new_image)
                
            new_exam['images'] = new_images  
            augmented_exams_set.append(exam)


RJ109662085


/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


8
RJ109696044
16
RJ109727291
24
RJ109729878


In [ ]:
print len(augmented_exams_set)

In [ ]:
pickle.dump(augmented_exams_set, open('augmented_images_baseline.pkl','w'), protocol=pickle.HIGHEST_PROTOCOL)

## Report Generation Model - Single Image Input

In [6]:
augmented_exams = pickle.load(open('augmented_images_baseline.pkl'))
print len(augmented_exams)

2154


Duplicate reports for each image

In [7]:
augmented_images = []
for exam in augmented_exams:
    for image in exam['images']:
        item = image
        item['tokens'] = exam['tokens']
        item['split'] = exam['split']
        if 'cnn features' not in item.keys():
            print exam['patient id']
        augmented_images.append(item)
print len(augmented_images)
print random.sample(augmented_images,1)

9812
[{'cnn features': array([[ 0.55561978,  0.        ,  0.        , ...,  0.        ,
         0.02577852,  0.        ]], dtype=float32), 'imid': 'CR.1.3.12.2.1107.5.4.4.1104.30000015021609065289000000091', 'tokens': [u'.', u'moderately', u'severe', u'tricompartmental', u'degenerative', u'change', u'bilaterally', u'marked', u'left', u'right', u'.', u'loose', u'within', u'knee', u'joint', u'left', '.'], 'impath': 'RJ109724280/CR.1.3.12.2.1107.5.4.4.1104.30000015021609065289000000091.jpg', 'split': 'train'}]


In [8]:
SEQUENCE_LENGTH = 33
MAX_SENTENCE_LENGTH = SEQUENCE_LENGTH - 3 # 1 for image, 1 for start token, 1 for end token
BATCH_SIZE = 20
CNN_FEATURE_SIZE = 1024
EMBEDDING_SIZE = 256

In [9]:
# Returns a list of tuples (cnn features, list of words, image ID)
def get_data_batch(dataset, size, split='train'):
    items = []
    
    while len(items) < size:
        item = random.choice(dataset)
        if item['split'] != split:
            continue
        sentence = item['tokens']
        if len(sentence) > MAX_SENTENCE_LENGTH:
            sentence = sentence[1:MAX_SENTENCE_LENGTH]
            #continue
        items.append((item['cnn features'], sentence, item['imid']))
    
    return items

In [10]:
# Convert a list of tuples into arrays that can be fed into the network
def prep_batch_for_network(batch):
    x_cnn = floatX(np.zeros((len(batch), CNN_FEATURE_SIZE)))
    x_sentence = np.zeros((len(batch), SEQUENCE_LENGTH - 1), dtype='int32')
    y_sentence = np.zeros((len(batch), SEQUENCE_LENGTH), dtype='int32')
    mask = np.zeros((len(batch), SEQUENCE_LENGTH), dtype='bool')

    for j, (cnn_features, sentence, _) in enumerate(batch):
        x_cnn[j] = cnn_features
        i = 0
        for word in ['#START#'] + sentence + ['#END#']:
            if word in word_to_index:
                mask[j, i] = True
                y_sentence[j, i] = word_to_index[word]
                x_sentence[j, i] = word_to_index[word]
                i += 1
        #mask[j, 0] = False
                
    return x_cnn, x_sentence, y_sentence, mask

In [11]:
# sentence embedding maps integer sequence with dim (BATCH_SIZE, SEQUENCE_LENGTH - 1) to 
# (BATCH_SIZE, SEQUENCE_LENGTH-1, EMBEDDING_SIZE)
l_input_sentence = lasagne.layers.InputLayer((BATCH_SIZE, SEQUENCE_LENGTH - 1))
l_sentence_embedding = lasagne.layers.EmbeddingLayer(l_input_sentence,
                                                     input_size=len(vocab),
                                                     output_size=EMBEDDING_SIZE,
                                                    )

# cnn embedding changes the dimensionality of the representation from 1024 to EMBEDDING_SIZE, 
# and reshapes to add the time dimension - final dim (BATCH_SIZE, 1, EMBEDDING_SIZE)
l_input_cnn = lasagne.layers.InputLayer((BATCH_SIZE, CNN_FEATURE_SIZE))
l_cnn_embedding = lasagne.layers.DenseLayer(l_input_cnn, num_units=EMBEDDING_SIZE,
                                            nonlinearity=lasagne.nonlinearities.identity)

l_cnn_embedding = lasagne.layers.ReshapeLayer(l_cnn_embedding, ([0], 1, [1]))

# the two are concatenated to form the RNN input with dim (BATCH_SIZE, SEQUENCE_LENGTH, EMBEDDING_SIZE)
l_rnn_input = lasagne.layers.ConcatLayer([l_cnn_embedding, l_sentence_embedding])


l_dropout_input = lasagne.layers.DropoutLayer(l_rnn_input, p=0.5)
l_lstm = lasagne.layers.LSTMLayer(l_dropout_input,
                                  num_units=EMBEDDING_SIZE,
                                  unroll_scan=True,
                                  grad_clipping=5.)
l_dropout_output = lasagne.layers.DropoutLayer(l_lstm, p=0.5)

# the RNN output is reshaped to combine the batch and time dimensions
# dim (BATCH_SIZE * SEQUENCE_LENGTH, EMBEDDING_SIZE)
l_shp = lasagne.layers.ReshapeLayer(l_dropout_output, (-1, EMBEDDING_SIZE))

# decoder is a fully connected layer with one output unit for each word in the vocabulary
l_decoder = lasagne.layers.DenseLayer(l_shp, num_units=len(vocab), nonlinearity=lasagne.nonlinearities.softmax)

# finally, the separation between batch and time dimension is restored
l_out = lasagne.layers.ReshapeLayer(l_decoder, (BATCH_SIZE, SEQUENCE_LENGTH, len(vocab)))

In [12]:
# cnn feature vector
x_cnn_sym = T.matrix()

# sentence encoded as sequence of integer word tokens
x_sentence_sym = T.imatrix()

# mask defines which elements of the sequence should be predicted
mask_sym = T.imatrix()

# ground truth for the RNN output
y_sentence_sym = T.imatrix()

In [13]:
output = lasagne.layers.get_output(l_out, {
                l_input_sentence: x_sentence_sym,
                l_input_cnn: x_cnn_sym
                })

In [14]:
def calc_cross_ent(net_output, mask, targets):
    # Helper function to calculate the cross entropy error
    preds = T.reshape(net_output, (-1, len(vocab)))
    targets = T.flatten(targets)
    cost = T.nnet.categorical_crossentropy(preds, targets)[T.flatten(mask).nonzero()]
    return cost

loss = T.mean(calc_cross_ent(output, mask_sym, y_sentence_sym))

In [16]:
MAX_GRAD_NORM = 15

all_params = lasagne.layers.get_all_params(l_out, trainable=True)

all_grads = T.grad(loss, all_params)
all_grads = [T.clip(g, -5, 5) for g in all_grads]
all_grads, norm = lasagne.updates.total_norm_constraint(
    all_grads, MAX_GRAD_NORM, return_norm=True)

updates = lasagne.updates.adam(all_grads, all_params, learning_rate=0.00001)

f_train = theano.function([x_cnn_sym, x_sentence_sym, mask_sym, y_sentence_sym],
                          [loss, norm],
                          updates=updates
                         )

f_val = theano.function([x_cnn_sym, x_sentence_sym, mask_sym, y_sentence_sym], loss)

In [17]:
for iteration in range(100000):
    x_cnn, x_sentence, y_sentence, mask = prep_batch_for_network(get_data_batch(augmented_images, BATCH_SIZE))
    loss_train, norm = f_train(x_cnn, x_sentence, mask, y_sentence)
    if not iteration % 250:
        print('Iteration {}, loss_train: {}, norm: {}'.format(iteration, loss_train, norm))
        try:
            batch = get_data_batch(augmented_images, BATCH_SIZE, split='val')
            x_cnn, x_sentence, y_sentence, mask = prep_batch_for_network(batch)
            loss_val = f_val(x_cnn, x_sentence, mask, y_sentence)
            print('Val loss: {}'.format(loss_val))
        except IndexError:
            continue

Iteration 0, loss_train: 5.12412261963, norm: 0.46596300602
Val loss: 5.12394857407
Iteration 250, loss_train: 4.92777585983, norm: 0.769301533699
Val loss: 4.89264774323
Iteration 500, loss_train: 4.26689529419, norm: 0.835332930088
Val loss: 4.4990439415
Iteration 750, loss_train: 4.24816560745, norm: 0.607361674309
Val loss: 4.37673950195
Iteration 1000, loss_train: 4.07771539688, norm: 0.605389893055
Val loss: 4.22698020935
Iteration 1250, loss_train: 3.99026179314, norm: 0.527254343033
Val loss: 4.27441883087
Iteration 1500, loss_train: 4.04906892776, norm: 0.503723561764
Val loss: 4.13545131683
Iteration 1750, loss_train: 3.94799566269, norm: 0.530963182449
Val loss: 3.97902870178
Iteration 2000, loss_train: 3.98307204247, norm: 0.561355173588
Val loss: 3.9612429142
Iteration 2250, loss_train: 3.95651817322, norm: 0.526784241199
Val loss: 4.01541805267
Iteration 2500, loss_train: 3.84807133675, norm: 0.502494096756
Val loss: 3.93274569511
Iteration 2750, loss_train: 3.85119819641

Iteration 23500, loss_train: 2.65410327911, norm: 2.10554003716
Val loss: 3.36878228188
Iteration 23750, loss_train: 2.56280064583, norm: 2.0782957077
Val loss: 3.45648026466
Iteration 24000, loss_train: 2.72524476051, norm: 2.10793423653
Val loss: 3.08804130554
Iteration 24250, loss_train: 2.56240940094, norm: 2.4070084095
Val loss: 3.25433349609
Iteration 24500, loss_train: 2.68536806107, norm: 2.4822883606
Val loss: 3.55413126945
Iteration 24750, loss_train: 2.48333764076, norm: 1.68692934513
Val loss: 3.24612569809
Iteration 25000, loss_train: 2.66186547279, norm: 1.58483219147
Val loss: 3.24375128746
Iteration 25250, loss_train: 2.46181941032, norm: 1.7790210247
Val loss: 3.18561530113
Iteration 25500, loss_train: 2.51687836647, norm: 1.95052230358
Val loss: 3.25824522972
Iteration 25750, loss_train: 2.51445817947, norm: 1.31469261646
Val loss: 3.05422019958
Iteration 26000, loss_train: 2.63461875916, norm: 1.73013651371
Val loss: 3.15489339828
Iteration 26250, loss_train: 2.68597

Iteration 47000, loss_train: 1.95230472088, norm: 2.50723147392
Val loss: 3.10752248764
Iteration 47250, loss_train: 2.05961847305, norm: 1.7025103569
Val loss: 3.47669529915
Iteration 47500, loss_train: 1.96740674973, norm: 1.65140354633
Val loss: 3.04024887085
Iteration 47750, loss_train: 2.09920716286, norm: 2.18202257156
Val loss: 3.06809091568
Iteration 48000, loss_train: 1.887134552, norm: 2.40102481842
Val loss: 3.03737330437
Iteration 48250, loss_train: 2.05409431458, norm: 1.6500825882
Val loss: 2.86211395264
Iteration 48500, loss_train: 1.86089587212, norm: 1.94330954552
Val loss: 2.95741629601
Iteration 48750, loss_train: 1.88908326626, norm: 2.01504135132
Val loss: 3.22826695442
Iteration 49000, loss_train: 1.9935516119, norm: 2.64654374123
Val loss: 2.94873213768
Iteration 49250, loss_train: 2.0845348835, norm: 1.60294640064
Val loss: 3.42038226128
Iteration 49500, loss_train: 1.94390499592, norm: 1.70454990864
Val loss: 2.91448545456
Iteration 49750, loss_train: 1.9301337

Iteration 70500, loss_train: 1.70494925976, norm: 2.18061685562
Val loss: 3.61847186089
Iteration 70750, loss_train: 1.54713523388, norm: 2.78348588943
Val loss: 3.77800893784
Iteration 71000, loss_train: 1.62787818909, norm: 1.95576369762
Val loss: 2.87939214706
Iteration 71250, loss_train: 1.42294609547, norm: 1.94410145283
Val loss: 3.64659714699
Iteration 71500, loss_train: 1.58204543591, norm: 1.97062647343
Val loss: 3.25859189034
Iteration 71750, loss_train: 1.61160755157, norm: 2.36363840103
Val loss: 3.2046983242
Iteration 72000, loss_train: 1.63012170792, norm: 2.0988676548
Val loss: 3.27167439461
Iteration 72250, loss_train: 1.66281986237, norm: 2.47841620445
Val loss: 3.08662438393
Iteration 72500, loss_train: 1.46624183655, norm: 1.70794594288
Val loss: 3.29423093796
Iteration 72750, loss_train: 1.58077764511, norm: 1.96272230148
Val loss: 3.04057359695
Iteration 73000, loss_train: 1.83151066303, norm: 2.10879015923
Val loss: 3.26036787033
Iteration 73250, loss_train: 1.554

KeyboardInterrupt: 

In [18]:
batch = get_data_batch(augmented_images, BATCH_SIZE, split='val')
x_cnn, x_sentence, y_sentence, mask = prep_batch_for_network(batch)
loss_val = f_val(x_cnn, x_sentence, mask, y_sentence)
print('Val loss: {}'.format(loss_val))

Val loss: 3.22634744644


In [19]:
param_values = lasagne.layers.get_all_param_values(l_out)
d = {'param values': param_values,
     'vocab': vocab,
     'word_to_index': word_to_index,
     'index_to_word': index_to_word,
    }
pickle.dump(d, open('aug_baseline_trained_v_167.pkl','w'), protocol=pickle.HIGHEST_PROTOCOL)